In [ ]:
#In this notebook, we will implement a baseline model in which we just predict October's sales for Novmber's sales 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Runs multiple output commands in the same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [65]:
%run -i 'DataLoader.py'

In [66]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [67]:
# We first group the data by months for every (shop_id, item_id) pair
monthly_data = train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day':'sum','item_price':'mean'}).reset_index(); # groupby automatically removes the column not mentioned, reset_index doesn't make the groupby columns as index
monthly_data = monthly_data.rename(columns={'date_block_num':'month_block_num','item_price':'mean_item_price','item_cnt_day':'item_cnt_month'});
monthly_data.sort_values(by=['shop_id','item_id','month_block_num'])

,shop_id,item_id,month_block_num,item_cnt_month,mean_item_price
0,0,30,1,31.0,265.0
1,0,31,1,11.0,434.0
2,0,32,0,6.0,221.0
3,0,32,1,10.0,221.0
4,0,33,0,3.0,347.0
...,...,...,...,...,...
1609119,59,22164,27,2.0,699.0
1609120,59,22164,30,1.0,699.0
1609121,59,22167,9,1.0,299.0
1609122,59,22167,11,2.0,299.0


In [70]:
# Taking only the (shop_id,item_id) pair having October data
last_month_data = monthly_data[monthly_data['month_block_num']==33]

# Clipping resultant values not in range 0 to 20 to reduce error (Check evaluation tab on Competition page)
last_month_data['item_cnt_month'] = last_month_data['item_cnt_month'].clip(0,20)

last_month_data

DataLoader.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items = pd.read_csv("KaggleData/items.csv")


,shop_id,item_id,month_block_num,item_cnt_month,mean_item_price
8113,2,31,33,1.0,399.0
8205,2,486,33,3.0,300.0
8254,2,787,33,1.0,420.0
8284,2,794,33,1.0,3300.0
8367,2,968,33,1.0,58.0
...,...,...,...,...,...
1608998,59,22087,33,6.0,119.0
1609030,59,22088,33,2.0,119.0
1609047,59,22091,33,1.0,179.0
1609073,59,22100,33,1.0,629.0


In [71]:
# Joining so that we include all test data points in the training set
baseline_sub = pd.merge(test,last_month_data,on=['shop_id','item_id'],how='left') 
# Creating submission file
baseline_sub = baseline_sub.drop(['shop_id','item_id','month_block_num','mean_item_price'],axis=1)
# Filling missing data with no sales
baseline_sub = baseline_sub.fillna(0)
baseline_sub

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,1.0
3,3,0.0
4,4,0.0
...,...,...
214195,214195,1.0
214196,214196,0.0
214197,214197,0.0
214198,214198,0.0


In [74]:
baseline_sub.to_csv('baseline_submission.csv',index=False) #index is False to not write index as a column in the csv file

In [77]:
baseline_sub

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,1.0
3,3,0.0
4,4,0.0
...,...,...
214195,214195,1.0
214196,214196,0.0
214197,214197,0.0
214198,214198,0.0
